In [6]:
import os
import openai
import csv
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Cargar las variables de entorno desde el archivo .env
load_dotenv(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\.env')

# Obtener la clave de API desde las variables de entorno
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API para OpenAI
openai.api_key = api_key
client = openai.Client()

def process_text_chunk_with_openai(text_chunk):
    """
    Procesa un chunk de texto usando la API de OpenAI y devuelve el resultado en formato CSV.
    """
    # Prompt específico para extraer tablas del texto proveniente de un cuadro de mando de Power BI
    final_prompt = (
        "Hola, tengo un archivo que contiene datos de texto sin estructurar, generado por la exportación de un cuadro de mando de Power BI. "
        "Me gustaría que identifiques y extraigas todas las tablas relevantes contenidas en este texto, ya que son datos tabulares generados por dicho cuadro de mando. "
        "Este archivo no tiene una estructura clara, pero las tablas son esenciales para entender la información de los informes. "
        "Aquí tienes un fragmento del texto que proviene del cuadro de mando de Power BI:"
        f"\n\n{text_chunk}\n\n"
        "Por favor, procesa este fragmento y devuelve las tablas relevantes de manera clara y formateadas en CSV, incluyendo filas y columnas bien definidas."
    )

    response = client.completions.create(
        
        model="gpt-4o-mini",  # Usar el modelo con soporte para 128,000 tokens
        prompt=final_prompt,
        max_tokens=15000,  # Reservamos 15,000 tokens para la respuesta
        temperature=0.5,
    )

    return response.choices[0].text.strip()

def split_text(text_data):
    """
    Divide los datos de texto en partes grandes para aprovechar el límite de tokens del modelo.
    """
    chunks = []
    max_chunk_size = 90000  # Tamaño máximo de cada chunk
    current_chunk = ""

    for char in text_data:
        current_chunk += char
        if len(current_chunk) >= max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = ""

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# Leer el archivo de texto completo
with open('PDF_extraido_layout.txt', 'r', encoding='utf-8') as file:
    full_text_data = file.read()

# Dividir el texto en partes manejables (chunks)
text_parts = split_text(full_text_data)

# Procesar todos los chunks en paralelo con progreso mostrado por tqdm
results = []
with ThreadPoolExecutor(max_workers=5) as executor:  # Ajusta el número de trabajadores según tus necesidades
    futures = [executor.submit(process_text_chunk_with_openai, chunk) for chunk in text_parts]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Procesando chunks"):
        results.append(future.result())

# Combinar todas las respuestas en una respuesta completa
combined_result = "\n".join(results)

# Guardar el resultado en formato CSV
with open("resultado_full.csv", "w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Tabla"])  # Si conoces las columnas de antemano, puedes adaptarlas aquí
    for result in results:
        writer.writerow([result])


Procesando chunks:   0%|          | 0/1 [00:00<?, ?it/s]


NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}